# Тестовое для СДЕК летней стажировки

Задача:
Предсказать цену (price) на аренду жилья, используя не только табличные данные, но и координаты.

Сслыка: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data/data

In [200]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

df = pd.read_csv("data/AB_NYC_2019.csv")

rows, cols = df.shape

print(f"Total rows: {rows}")
print(f"Total columns: {cols}")

Total rows: 48895
Total columns: 16


In [201]:
print(df.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


Колонки для анализа:
- __host_id__ - уникальные id владельца (заменим им host_name)
- __neighbourhood_group__ - боро, где расположено помещение (боро (borough) - горада Нью-Йорка, основная административная единица)
- __neighbourhood__ - районы внутри боро
- __root_type__ - тип помещения
- __minimum_nights__ - минимальное количество ночей для аренды помещения
- __number_of_reviews__ - общее количество отзывов
- __last_review__ - дата последнего отзыва
- __reviews_per_month__ - срднее количество отзывов за месяц
- __calculated_host_listings_count__ - количество объявлений от хозяина
- __availability_365__ - количество дней в году, когда жильё доступно
 Остальные колонки будут отброшены.

In [202]:
# удаляем ненужные колонки
df = df.drop(["id", "name", "host_name"], axis=1)

print(df.columns)

Index(['host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')


In [203]:
ids, indexes = np.unique(df["host_id"], return_index=True)
mappings = dict(zip(ids, indexes))

df["host_id"] = df["host_id"].map(mappings)
df["host_id"] /= df["host_id"].max()

In [204]:
mask = (0 <= df["availability_365"]) & (df["availability_365"] <= 365)
df = df[mask]
df["availability_365"] /= 365

In [205]:
labels_to_one_hot = ["neighbourhood_group", "neighbourhood", "room_type"]
for label in labels_to_one_hot:
    one_hot = pd.get_dummies(df[label], dtype=np.float32)
    df = df.join(one_hot)
df = df.drop(labels_to_one_hot, axis=1)

In [ ]:
df["reviews_per_month"] = df["reviews_per_month"].fillna(0)

min_date = pd.to_datetime(df["last_review"]).min()
df["last_review"] = pd.to_datetime(df["last_review"].fillna(min_date))
df["last_review"] = df["last_review"].max() - df["last_review"]
df["last_review"] = df["last_review"].apply((lambda date: date.days))
df["last_review"] /= df["last_review"].max()

0        0.086640
1        0.015873
2        1.000000
3        0.000992
4        0.076389
           ...   
48890    1.000000
48891    1.000000
48892    1.000000
48893    1.000000
48894    1.000000
Name: last_review, Length: 48895, dtype: float64


In [207]:
for label in ["minimum_nights", "reviews_per_month", "calculated_host_listings_count"]:
    mask = df[label] >= 0
    df = df[mask]
    df[label] /= df[label].max()

In [ ]:
df.to_csv('normalized.csv', index=False)